# 6章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
# 6-1
!mkdir chap6
%cd ./chap6

/home/root/work/chap6


In [4]:
# 6-2
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

In [5]:
# 6-3
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [4]:
# 6-4
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)
bert_sc = bert_sc.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [5]:
# 6-5
text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。"
]
label_list = [1,0,1]

# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items() }
labels = torch.tensor(label_list).cuda()

# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1) # スコアが最も高いラベル
num_correct = (labels_predicted==labels).sum().item() # 正解数
accuracy = num_correct/labels.size(0) # 精度

print("# scores:")
print(scores.size())
print("# predicted labels:")
print(labels_predicted)
print("# accuracy:")
print(accuracy)

# scores:
torch.Size([3, 2])
# predicted labels:
tensor([0, 0, 0], device='cuda:0')
# accuracy:
0.3333333333333333


In [6]:
# 6-6
# 符号化
encoding = tokenizer(
    text_list, 
    padding='longest',
    return_tensors='pt'
) 
encoding['labels'] = torch.tensor(label_list) # 入力にラベルを加える。
encoding = { k: v.cuda() for k, v in encoding.items() }

# ロスの計算
output = bert_sc(**encoding)
loss = output.loss # 損失の取得
print(loss)

tensor(0.7716, device='cuda:0', grad_fn=<NllLossBackward0>)


In [8]:
# 6-7
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 
#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz 

--2022-05-08 17:57:50--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  3.49MB/s    in 2.4s    

2022-05-08 17:57:54 (3.49 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [9]:
# 6-8
!cat ./text/it-life-hack/it-life-hack-6342280.txt # ファイルを表示

http://news.livedoor.com/article/detail/6342280/
2012-03-06T13:00:00+0900
USB3.0対応で爆速データ転送！　9倍速のリーダー／ライター登場
USB3.0が登場してから今年で4年目となるがパソコン側でのUSB3.0ポート搭載が進んで来ても対応機器がなかなか充実していない現状がある。そんな中で新しく高速な読み取りが可能なメモリーカードリーダー／ライターが登場した。

バッファローコクヨサプライがUSB3.0対応のカードリーダー／ライターを発表した。SDHC対応のSD系メディアやコンパクトフラッシュ、メモリースティック系メディア、xDピクチャーカードといったデジカメやスマホ、携帯ゲームといった機器で使われている各種メディアを従来よりも短時間でPCに取り込むことが可能になる。

転送速度が5Gbps（理論値）とUSB2.0の480Mbpsと比べて爆速になったUSB3.0はPC側の対応が進んで来ていたが高速転送が生かせる周辺機器としては、外付けHDDや一部のUSBメモリーくらいしかなかった。これに多くのメディアが扱えるリーダー／ライターが加わることで手軽にUSB3.0の恩恵を受けることができるようになる。

今回発表されたのは、USB3.0ケーブルとカードリーダー本体が分かれるタイプの「BSCR09U3」シリーズ（3,240円）、USB3.0コネクタをカードリーダー本体に内蔵している「BSCRD04U3」シリーズ（2,690円）だ。共にホワイトとブラックのカラーバリエーションが用意される（発売は3月下旬以降）。

■リリースページ
■バッファローコクヨサプライ




■バッファローの記事をもっと見る
・約283gでカバンに入る！小型キーボードの驚くべき機能
・3種類のホットキーで使いやすい！AndroidとPCで使えるキーボードの魅力
・ドラえもんもビックリの新アイテム！マウスとキーボードが合体"OPAir"
・ありそうでなかった便利機能！ファイル仕分けする画期的なHDD


サンディスク SanDisk microSDHC 32GB（microSD 32GB） 超高速クラス4  変換アダプター付 世界国内シェアNo.1 バルク品
クチコミを見る


In [10]:
# 6-9
# データローダーの作成
dataset_for_loader = [
    {'data':torch.tensor([0,1]), 'labels':torch.tensor(0)},
    {'data':torch.tensor([2,3]), 'labels':torch.tensor(1)},
    {'data':torch.tensor([4,5]), 'labels':torch.tensor(2)},
    {'data':torch.tensor([6,7]), 'labels':torch.tensor(3)},
]
loader = DataLoader(dataset_for_loader, batch_size=2)

# データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)
    ## ファインチューニングではここでミニバッチ毎の処理を行う

# batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
# batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}


In [11]:
# 6-10
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)

for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

# batch 0
{'data': tensor([[6, 7],
        [2, 3]]), 'labels': tensor([3, 1])}
# batch 1
{'data': tensor([[0, 1],
        [4, 5]]), 'labels': tensor([0, 2])}


In [12]:
# 6-11
# カテゴリーのリスト
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 各データの形式を整える
max_length = 128
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./text/{category}/{category}*'):
        lines = open(file).read().splitlines()
        text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す。
        encoding = tokenizer(
            text,
            max_length=max_length, 
            padding='max_length',
            truncation=True
        )
        encoding['labels'] = label # ラベルを追加
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)

100%|██████████| 9/9 [00:23<00:00,  2.64s/it]


In [13]:
# 6-12
print(dataset_for_loader[0])

{'input_ids': tensor([    2,    63, 12455, 15060,   598, 28469,    80,   630,     8,  3904,
          788,   640,  5654,  7134,     5,    54,    65,  2198,    52, 29760,
           12,   335,  1950,    26,     5,   103, 28719, 28485,    28,    80,
        18111,  4358,   762,  9263,  1150, 28472, 15431,    23, 10703, 28485,
        28472,  5825,    93,    24,   525,   201,   569,   335,     8,  4799,
         1450,    14,     6,  3829,  4358,     5,    82,     6,  2203,  5385,
         1269,  1964,     7,   651,    15,     6,    42,  7305,    11, 24075,
           23,  8973,    28, 17372,    28,    23, 29805, 28473, 28473,   140,
        13368,    35,  6486,     8, 14163, 14650,     5, 12479,    23,  5749,
        28472,  9543, 28459,    24,    14,     6,  9263,  1150,    11, 19702,
           12, 14710,     7,    15,    16,  1497,  4367,     9,     6,    36,
         1728,    35,  8758,    35, 11374,    38,    49,    36,  3898,   158,
        28472, 27920,   764,    38,     5,   124, 

In [14]:
# 6-13
# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [15]:
# 6-14
class BertForSequenceClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [16]:
# 6-15
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    callbacks = [checkpoint]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
# 6-16
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val) 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [18]:
# 6-17
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

ベストモデルのファイル:  /home/root/work/chap6/model/epoch=3-step=556.ckpt
ベストモデルの検証データに対する損失:  tensor(0.4296, device='cuda:0')


In [6]:
# 6-18
%reload_ext tensorboard
%tensorboard --logdir ./ --port 8081 --host 0.0.0.0

Reusing TensorBoard on port 8081 (pid 73), started 0:00:10 ago. (Use '!kill 73' to kill it.)

In [20]:
# 6-19
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1444: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /home/root/work/chap6/model/epoch=3-step=556.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/root/work/chap6/model/epoch=3-step=556.ckpt
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  ran

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.8860244154930115
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Accuracy: 0.89


In [21]:
# 6-20
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
) 

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained('./model_transformers') 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [22]:
# 6-21
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_transformers'
)